<a href="https://colab.research.google.com/github/DKarev/kombu/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install openai
!pip install chromedriver-autoinstaller
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 38.4 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [52.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,077 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,755 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:11 https://ppa.launchpadcontent.net/de

In [43]:
from bs4 import BeautifulSoup
import requests
import os
import openai
from openai import OpenAI
import sys
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

import chromedriver_autoinstaller

In [44]:
# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

In [8]:
test_url = 'https://demo.evershop.io/'

In [37]:
if test_url[-1] == '/':
  test_url = test_url[:-1]

In [9]:
with open('navigation_path_20240427_140124.424242.txt', 'r') as file:
    testFile = file.read()
    print(testFile)

Node: Home	Full URL: /
Node: Men	Full URL: /men
Node: Nike React Phantom Run Flyknit 2	Full URL: /men/nike-react-phantom-run-flyknit-2-179
Node: Size X	Full URL: /men/nike-react-phantom-run-flyknit-2-179?size=4
Node: Color Green	Full URL: /men/nike-react-phantom-run-flyknit-2-179?size=4&color=17
Node: Cart	Full URL: /cart
Node: Home	Full URL: /
Node: Women	Full URL: /women
Node: Alphaedge 4d Reflective Shoes R	Full URL: /women/alphaedge-4d-reflective-shoes-23
Node: Size XL	Full URL: /women/alphaedge-4d-reflective-shoes-23?size=26
Node: Color White	Full URL: /women/alphaedge-4d-reflective-shoes-23?size=26&color=18
Node: Cart	Full URL: /cart
Node: Checkout	Full URL: /cart/checkout


In [10]:
testLines = testFile.split('\n')

In [21]:
all_nodes = []
all_urls = []
for x in testLines:
  node, fullUrl = x.split('\t')
  node = node[6:]
  fullUrl = fullUrl[10:]
  all_nodes.append(node)
  all_urls.append(fullUrl)
  print(node, fullUrl)

Home /
Men /men
Nike React Phantom Run Flyknit 2 /men/nike-react-phantom-run-flyknit-2-179
Size X /men/nike-react-phantom-run-flyknit-2-179?size=4
Color Green /men/nike-react-phantom-run-flyknit-2-179?size=4&color=17
Cart /cart
Home /
Women /women
Alphaedge 4d Reflective Shoes R /women/alphaedge-4d-reflective-shoes-23
Size XL /women/alphaedge-4d-reflective-shoes-23?size=26
Color White /women/alphaedge-4d-reflective-shoes-23?size=26&color=18
Cart /cart
Checkout /cart/checkout


In [38]:
start_node = test_url + all_urls[0]

In [80]:
def scrape_html (url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  #all_elements = soup.find_all(True)  # Find all elements
  return soup

In [ ]:
html_code[0]

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

In [125]:
start_node

'https://demo.evershop.io/men/nike-react-phantom-run-flyknit-2-179?size=4&color=17'

In [201]:
html_code = scrape_html(start_node)

In [202]:
len(html_code.prettify())

13438

In [160]:
all_urls[6].split('/')[-1]

''

In [203]:
dest = all_urls[12].split('/')[-1]
dest

'checkout'

In [223]:
start_node = test_url + all_urls[0]

In [224]:
for url in all_urls:
  dest = url.split('/')[-1]
  html_code = scrape_html(start_node)
  prompt = """ Task: Given HTML code for a web page, please provide a few Selenium code lines
that would allow us to click on the button that goes to the page that we will specify.
Provide only the relevent lines of code without any additional explanations.

Examples without HTML being provided:

1. "How can I get to /men page?"
Answer:
shopMen = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Shop men']")
shopMen.click()

2. "How can I get to /hacked-fashion-chuck-taylor-all-star-85 page?"
Answer:
hackyShoes = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Hacked fashion chuck taylor all star']")
hackyShoes.click()

3. "How can I get to /hacked-fashion-chuck-taylor-all-star-85?size=26 page?"
Answer:
sizeSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='XL']")
sizeSelect.click()

4. "How can I get to /hacked-fashion-chuck-taylor-all-star-85?size=26&color=8 page?"
Answer:
colorSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='Blue']")
colorSelect.click()


5. "How can I get to /cart page?"
Answer:
addToCart = driver.find_element(by=By.XPATH, value="//button[@type='button']")
addToCart.click()

If you ever encounter a mandatory field on the html, you have to provide the Selenium code that writes something in those fields.
More specifically, you will encounter those on the cart page when trying to get to the /checkout page. If asked how to get to '/checkout' page,
you should provide the answer EXACTLY as written in example 6.

6. "How can I get to /checkout page?"
Answer:

viewCart = driver.find_element(by=By.XPATH, value="//a[normalize-space()='VIEW CART (1)']")
viewCart.click()

checkOut = driver.find_element(by=By.XPATH, value="//span[normalize-space()='CHECKOUT']")
checkOut.click()

emailBox = driver.find_element(by=By.XPATH, value="//input[@placeholder='Email']")
emailBox.send_keys("KombuAITest@PearHack.com")

continueToShipping = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Continue to shipping']")
continueToShipping.click()

fullName = driver.find_element(by=By.XPATH, value="//input[@placeholder='Full name']")
phone = driver.find_element(by=By.XPATH, value="//input[@placeholder='Telephone']")
address = driver.find_element(by=By.XPATH, value="//input[@placeholder='Address']")
city = driver.find_element(by=By.XPATH, value="//input[@placeholder='City']")
country = driver.find_element(by=By.XPATH, value="//select[contains(@id,'address')]")
postcode = driver.find_element(by=By.XPATH, value="//input[@placeholder='Postcode']")

fullName.send_keys("AiTest")
phone.send_keys("AiTest")
address.send_keys("AiTest")
city.send_keys("San Francisco")
postcode.send_keys("94107")

select = Select(country)
select.select_by_visible_text('United States')

province = driver.find_element(by=By.XPATH, value="(//select[@id='address[province]'])[1]")
selectProvince = Select(province)
selectProvince.select_by_visible_text('California')

shipping = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Standard Delivery - $5.00']")
shipping.click()

toPayment = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Continue to payment']")
toPayment.click()

cashOnDelivery = driver.find_element(by=By.XPATH, value="//div[@class='checkout-payment checkout-step']//div//div[1]//div[1]//div[1]//div[1]//div[1]//a[1]//*[name()='svg']")
cashOnDelivery.click()

placeOrder = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Place Order']")
placeOrder.click()


Based on these examples and based on the provided HTML code, please answer the following prompt:
How can I get to /{}?
Here is the HTML: {}
""".format(dest, html_code.prettify())
  stream = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)
  for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")
  start_node = test_url + url



```python
logoIcon = driver.find_element(by=By.XPATH, value="//a[@class='logo-icon']")
logoIcon.click()
```shopMen = driver.find_element(by=By.XPATH, value="//a[@class='nav-link hover:underline'][@href='/men']")
shopMen.click()reactPhantomFlyknit = driver.find_element(by=By.XPATH, value="//a[@href='/men/nike-react-phantom-run-flyknit-2-179']")
reactPhantomFlyknit.click()sizeSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='X']")
sizeSelect.click()sizeSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='X']")
sizeSelect.click()

colorSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='Green']")
colorSelect.click()addToCart = driver.find_element(by=By.XPATH, value="//a[@class='mini-cart-icon']")
addToCart.click()homeLink = driver.find_element(by=By.XPATH, value="//a[@class='logo-icon']")
homeLink.click()```python
womenLink = driver.find_element(by=By.XPATH, value="//a[normalize-space()='Women']")
womenLink.click()
```alphaEdgeS

In [217]:
prompt = """ Task: Given HTML code for a web page, please provide a few Selenium code lines
that would allow us to click on the button that goes to the page that we will specify.
Provide only the relevent lines of code without any additional explanations.

Examples without HTML being provided:

1. "How can I get to /men page?"
Answer:
shopMen = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Shop men']")
shopMen.click()

2. "How can I get to /hacked-fashion-chuck-taylor-all-star-85 page?"
Answer:
hackyShoes = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Hacked fashion chuck taylor all star']")
hackyShoes.click()

3. "How can I get to /hacked-fashion-chuck-taylor-all-star-85?size=26 page?"
Answer:
sizeSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='XL']")
sizeSelect.click()

4. "How can I get to /hacked-fashion-chuck-taylor-all-star-85?size=26&color=8 page?"
Answer:
colorSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='Blue']")
colorSelect.click()


5. "How can I get to /cart page?"
Answer:
addToCart = driver.find_element(by=By.XPATH, value="//button[@type='button']")
addToCart.click()

If you ever encounter a mandatory field on the html, you have to provide the Selenium code that writes something in those fields.
More specifically, you will encounter those on the cart page when trying to get to the /checkout page. If asked how to get to '/checkout' page,
you should provide the answer EXACTLY as written in example 6.

6. "How can I get to /checkout page?"
Answer:

viewCart = driver.find_element(by=By.XPATH, value="//a[normalize-space()='VIEW CART (1)']")
viewCart.click()

checkOut = driver.find_element(by=By.XPATH, value="//span[normalize-space()='CHECKOUT']")
checkOut.click()

emailBox = driver.find_element(by=By.XPATH, value="//input[@placeholder='Email']")
emailBox.send_keys("KombuAITest@PearHack.com")

continueToShipping = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Continue to shipping']")
continueToShipping.click()

fullName = driver.find_element(by=By.XPATH, value="//input[@placeholder='Full name']")
phone = driver.find_element(by=By.XPATH, value="//input[@placeholder='Telephone']")
address = driver.find_element(by=By.XPATH, value="//input[@placeholder='Address']")
city = driver.find_element(by=By.XPATH, value="//input[@placeholder='City']")
country = driver.find_element(by=By.XPATH, value="//select[contains(@id,'address')]")
postcode = driver.find_element(by=By.XPATH, value="//input[@placeholder='Postcode']")

fullName.send_keys("AiTest")
phone.send_keys("AiTest")
address.send_keys("AiTest")
city.send_keys("San Francisco")
postcode.send_keys("94107")

select = Select(country)
select.select_by_visible_text('United States')

province = driver.find_element(by=By.XPATH, value="(//select[@id='address[province]'])[1]")
selectProvince = Select(province)
selectProvince.select_by_visible_text('California')

shipping = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Standard Delivery - $5.00']")
shipping.click()

toPayment = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Continue to payment']")
toPayment.click()

cashOnDelivery = driver.find_element(by=By.XPATH, value="//div[@class='checkout-payment checkout-step']//div//div[1]//div[1]//div[1]//div[1]//div[1]//a[1]//*[name()='svg']")
cashOnDelivery.click()

placeOrder = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Place Order']")
placeOrder.click()


Based on these examples and based on the provided HTML code, please answer the following prompt:
How can I get to /{}?
Here is the HTML: {}
""".format(dest, html_code.prettify())


In [218]:
stream = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)

In [219]:
for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

```python
addToCart = driver.find_element(by=By.XPATH, value="//button[@type='button']")
addToCart.click()

viewCart = driver.find_element(by=By.XPATH, value="//a[normalize-space()='VIEW CART (1)']")
viewCart.click()

checkOut = driver.find_element(by=By.XPATH, value="//span[normalize-space()='CHECKOUT']")
checkOut.click()

emailBox = driver.find_element(by=By.XPATH, value="//input[@placeholder='Email']")
emailBox.send_keys("KombuAITest@PearHack.com")

continueToShipping = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Continue to shipping']")
continueToShipping.click()

fullName = driver.find_element(by=By.XPATH, value="//input[@placeholder='Full name']")
phone = driver.find_element(by=By.XPATH, value="//input[@placeholder='Telephone']")
address = driver.find_element(by=By.XPATH, value="//input[@placeholder='Address']")
city = driver.find_element(by=By.XPATH, value="//input[@placeholder='City']")
country = driver.find_element(by=By.XPATH, value="//select[contains(@i

In [131]:
driver = webdriver.Chrome(options=chrome_options)
driver.get(start_node)

In [98]:
start_node

'https://demo.evershop.io/men/nike-react-phantom-run-flyknit-2-179'

In [61]:
shopMen = driver.find_element(by=By.XPATH, value="//a[normalize-space()='Men']")
shopMen.click()

In [90]:
nikeReactPhantom = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Nike react phantom run flyknit 2']")
nikeReactPhantom.click()

In [105]:
sizeSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='X']")
sizeSelect.click()

In [120]:
colorSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='Green']")
colorSelect.click()

In [132]:
cartButton = driver.find_element(by=By.XPATH, value="//a[@href='/cart']")
cartButton.click()


In [144]:
home = driver.find_element(by=By.XPATH, value="//a[@class='logo-icon']")
home.click()

In [154]:
women = driver.find_element(by=By.XPATH, value="//a[normalize-space()='Women']")
women.click()

In [165]:
alphaedgeShoes = driver.find_element(by=By.XPATH, value="//a[@href='/women/alphaedge-4d-reflective-shoes-23']")
alphaedgeShoes.click()

In [176]:
sizeSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='XL']")
sizeSelect.click()

In [187]:
sizeSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='XL']")
sizeSelect.click()

colorSelect = driver.find_element(by=By.XPATH, value="//a[normalize-space()='White']")
colorSelect.click()

In [197]:
cart = driver.find_element(by=By.XPATH, value="//a[@href='/cart']")
cart.click()

In [220]:
viewCart = driver.find_element(by=By.XPATH, value="//a[normalize-space()='VIEW CART (1)']")
viewCart.click()

checkOut = driver.find_element(by=By.XPATH, value="//span[normalize-space()='CHECKOUT']")
checkOut.click()

emailBox = driver.find_element(by=By.XPATH, value="//input[@placeholder='Email']")
emailBox.send_keys("KombuAITest@PearHack.com")

continueToShipping = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Continue to shipping']")
continueToShipping.click()

fullName = driver.find_element(by=By.XPATH, value="//input[@placeholder='Full name']")
phone = driver.find_element(by=By.XPATH, value="//input[@placeholder='Telephone']")
address = driver.find_element(by=By.XPATH, value="//input[@placeholder='Address']")
city = driver.find_element(by=By.XPATH, value="//input[@placeholder='City']")
country = driver.find_element(by=By.XPATH, value="//select[contains(@id,'address')]")
postcode = driver.find_element(by=By.XPATH, value="//input[@placeholder='Postcode']")

fullName.send_keys("AiTest")
phone.send_keys("AiTest")
address.send_keys("AiTest")
city.send_keys("San Francisco")
postcode.send_keys("94107")

select = Select(country)
select.select_by_visible_text('United States')

province = driver.find_element(by=By.XPATH, value="(//select[@id='address[province]'])[1]")
selectProvince = Select(province)
selectProvince.select_by_visible_text('California')

shipping = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Standard Delivery - $5.00']")
shipping.click()

toPayment = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Continue to payment']")
toPayment.click()

cashOnDelivery = driver.find_element(by=By.XPATH, value="//div[@class='checkout-payment checkout-step']//div//div[1]//div[1]//div[1]//div[1]//div[1]//a[1]//*[name()='svg']")
cashOnDelivery.click()

placeOrder = driver.find_element(by=By.XPATH, value="//span[normalize-space()='Place Order']")
placeOrder.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@type='button']"}
  (Session info: chrome-headless-shell=124.0.6367.91); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5b07d7217cb3 <unknown>
#1 0x5b07d6f064a7 <unknown>
#2 0x5b07d6f5131d <unknown>
#3 0x5b07d6f513d1 <unknown>
#4 0x5b07d6f94734 <unknown>
#5 0x5b07d6f7343d <unknown>
#6 0x5b07d6f91be9 <unknown>
#7 0x5b07d6f731b3 <unknown>
#8 0x5b07d6f4424a <unknown>
#9 0x5b07d6f44c1e <unknown>
#10 0x5b07d71dbfdb <unknown>
#11 0x5b07d71dff2b <unknown>
#12 0x5b07d71c80f1 <unknown>
#13 0x5b07d71e0a92 <unknown>
#14 0x5b07d71acfaf <unknown>
#15 0x5b07d7206e08 <unknown>
#16 0x5b07d7206fe0 <unknown>
#17 0x5b07d7216e04 <unknown>
#18 0x7b882c8d0ac3 <unknown>


In [198]:
dest

'cart'

In [213]:
driver.current_url

'https://demo.evershop.io/cart'

In [200]:
start_node = driver.current_url

In [215]:
#fullName = driver.find_element(by=By.XPATH, value="//input[@placeholder='Full name']")
fullName = driver.find_element(by=By.XPATH, value="//input[@placeholder='Full name']")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//input[@placeholder='Full name']"}
  (Session info: chrome-headless-shell=124.0.6367.91); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5b07d7217cb3 <unknown>
#1 0x5b07d6f064a7 <unknown>
#2 0x5b07d6f5131d <unknown>
#3 0x5b07d6f513d1 <unknown>
#4 0x5b07d6f94734 <unknown>
#5 0x5b07d6f7343d <unknown>
#6 0x5b07d6f91be9 <unknown>
#7 0x5b07d6f731b3 <unknown>
#8 0x5b07d6f4424a <unknown>
#9 0x5b07d6f44c1e <unknown>
#10 0x5b07d71dbfdb <unknown>
#11 0x5b07d71dff2b <unknown>
#12 0x5b07d71c80f1 <unknown>
#13 0x5b07d71e0a92 <unknown>
#14 0x5b07d71acfaf <unknown>
#15 0x5b07d7206e08 <unknown>
#16 0x5b07d7206fe0 <unknown>
#17 0x5b07d7216e04 <unknown>
#18 0x7b882c8d0ac3 <unknown>
